### Link gitlab
https://romeogit.univ-reims.fr/mnoizet/visualisationscientifique

#### Implémentation détaillée

1. **volume.cu**  
   - Dupliquer la fonction `mip()` pour créer `minip()` (minimum intensity projection) et `meanip()` (mean intensity projection).
   - Adapter la logique pour chaque fonction :
     - `minip` : cherche la plus petite intensité sur le rayon.
     - `meanip` : calcule la moyenne des intensités valides sur le rayon.
   - Dans `__closesthit__volume_radiance`, utiliser un `switch` sur `optixLaunchParams.frame.renderType` pour appeler la bonne fonction.

2. **LaunchParams.h**  
   - Ajouter `MINIP` et `MEANIP` à l'enum `RENDER_TYPE`.

3. **screenDisplay.cpp**  
   - Ajouter des boutons radio (ligne ~194) pour permettre à l'utilisateur de choisir le mode de rendu (MIP, MINIP, MEANIP) via l'interface ImGui.
   - Modifier la valeur de `parameters->frame.renderType` selon le choix utilisateur.

#### Résultat du filtre MINIP

Affichage du rendu obtenu avec le filtre MINIP :

![Filtre MINIP](capture/MINIP.png)

#### Résultat du filtre MEANIP

Affichage du rendu obtenu avec le filtre MEANIP :

![Filtre MEANIP](capture/MEANIP.png)

#### Code modifié / ajouté

##### VisualisationScientifique/shader/volume.cu
```cuda
// filepath: /home/corentin/Documents/Cours/M2/Projets/RenduVisuLaurantLucas/TP2/VisualisationScientifique/shader/volume.cu
// ...existing code...

  __device__ void minip(){
      // ...voir code complet dans le fichier...
  }

  __device__ void meanip(){
      // ...voir code complet dans le fichier...
  }

  extern "C" __global__ void __closesthit__volume_radiance(){
      switch(optixLaunchParams.frame.renderType) {
        case 0: // MIP
          mip();
          break;
        case 1: // MINIP
          minip();
          break;
        case 2: // MEANIP
          meanip();
          break;
        default:
          mip();
          break;
      }
  }
// ...existing code...
```

##### VisualisationScientifique/common/LaunchParams.h
```cpp
// filepath: /home/corentin/Documents/Cours/M2/Projets/RenduVisuLaurantLucas/TP2/VisualisationScientifique/common/LaunchParams.h
// ...existing code...
enum RENDER_TYPE{MIP, MINIP, MEANIP};
// ...existing code...
```

##### VisualisationScientifique/common/screenDisplay.cpp
```cpp
// filepath: /home/corentin/Documents/Cours/M2/Projets/RenduVisuLaurantLucas/TP2/VisualisationScientifique/common/screenDisplay.cpp
// ...existing code...
    // Ajout des boutons radio pour le type de rendu
    static int renderType = parameters->frame.renderType;
    ImGui::Text("Render Type:");
    ImGui::RadioButton("MIP", &renderType, 0); ImGui::SameLine();
    ImGui::RadioButton("MINIP", &renderType, 1); ImGui::SameLine();
    ImGui::RadioButton("MEANIP", &renderType, 2);
    parameters->frame.renderType = renderType;
// ...existing code...
```

#### Ajout du mode DVR (Direct Volume Rendering)

1. **volume.cu**  
   - Ajouter la fonction `dvr()` pour le rendu volume direct.
   - Étendre le switch dans `__closesthit__volume_radiance` pour gérer le cas DVR.

2. **LaunchParams.h**  
   - Ajouter `DVR` à l'enum `RENDER_TYPE`.

3. **screenDisplay.cpp**  
   - Ajouter un bouton radio "DVR" pour permettre la sélection du mode via l'interface ImGui.

---

##### VisualisationScientifique/shader/volume.cu
```cuda
// ...existing code...

  __device__ void dvr(){
      // Implémentation du rendu volume direct (DVR)
      // ...voir code complet dans le fichier...
  }

  extern "C" __global__ void __closesthit__volume_radiance(){
      switch(optixLaunchParams.frame.renderType) {
        case 0: // MIP
          mip();
          break;
        case 1: // MINIP
          minip();
          break;
        case 2: // MEANIP
          meanip();
          break;
        case 3: // DVR
          dvr();
          break;
        default:
          mip();
          break;
      }
  }
// ...existing code...
```

##### VisualisationScientifique/common/LaunchParams.h
```cpp
// ...existing code...
enum RENDER_TYPE{MIP, MINIP, MEANIP, DVR};
// ...existing code...
```

##### VisualisationScientifique/common/screenDisplay.cpp
```cpp
// ...existing code...
    // Ajout des boutons radio pour le type de rendu
    static int renderType = parameters->frame.renderType;
    ImGui::Text("Render Type:");
    ImGui::RadioButton("MIP", &renderType, 0); ImGui::SameLine();
    ImGui::RadioButton("MINIP", &renderType, 1); ImGui::SameLine();
    ImGui::RadioButton("MEANIP", &renderType, 2); ImGui::SameLine();
    ImGui::RadioButton("DVR", &renderType, 3);
    parameters->frame.renderType = renderType;
// ...existing code...
```


#### Transfert d'opacité et de couleur pour le mode DVR

Pour le rendu volume direct (DVR), on utilise une fonction de transfert qui associe à chaque intensité du volume une couleur et une opacité. Cela permet de révéler différentes structures internes selon leur valeur d'intensité.

Exemple de fonction de transfert simple (dans le code CUDA) :

```cuda
__device__ vec3f transferFunction(float intensity, float& alpha) {
    // Couleur en niveaux de gris, opacité croissante avec l'intensité
    vec3f color = vec3f(intensity, intensity, intensity); // gris
    alpha = fminf(fmaxf((intensity - optixLaunchParams.frame.minIntensity) /
                        (optixLaunchParams.frame.maxIntensity - optixLaunchParams.frame.minIntensity), 0.0f), 1.0f);
    return color;
}
```

Dans la fonction `dvr()`, à chaque échantillon sur le rayon :
- On récupère l'intensité du voxel.
- On applique la fonction de transfert pour obtenir la couleur et l'opacité.
- On accumule la couleur en tenant compte de l'opacité déjà accumulée (compositing avant-arrière).

Ce principe permet d'obtenir un rendu semi-transparent où les structures internes du volume sont visibles selon leur intensité et la fonction de transfert choisie.

Vous pouvez personnaliser la fonction de transfert pour mettre en valeur certaines plages d'intensité ou utiliser des couleurs différentes selon les besoins de visualisation.